# PSG Diaspora Dataset - Data Exploration

This notebook explores the collected data about Île-de-France football players.

In [ ]:
import pandas as pd
import plotly.express as px
import json
from pathlib import Path

# Project root
PROJECT_ROOT = Path("..")
DATA_DIR = PROJECT_ROOT / "data"

## 1. Load Data

Load the collected player data from JSON files.

In [ ]:
# TODO: Load data after collection
# Example:
# with open(DATA_DIR / "raw" / "transfermarkt" / "psg_academy_players.json") as f:
#     players = json.load(f)
# df = pd.DataFrame(players)
# df.head()

## 2. Geographic Distribution

Map the birthplaces of players to understand geographic concentration.

In [ ]:
# TODO: Create map visualization
# Example with folium or plotly
pass

## 3. Department Analysis

Which départements produce the most players?

In [ ]:
# TODO: Analyze by department (75, 91, 92, 93, 94, 95, 77, 78)

## 4. Career Trajectories

Where do Île-de-France players end up playing?

In [ ]:
# TODO: Sankey diagram of career paths

## 5. Comparison with Demographics

Correlate player production with demographic data.

In [ ]:
# TODO: Load INSEE data and correlate